In [2]:
import glob
import os
import pandas as pd
import sys
import json
import tqdm
import numpy as np
import itertools
import seaborn as sns
!pip install statsmodels
import statsmodels.api as sm

    100% |████████████████████████████████| 10.5MB 137kB/s ta 0:00:01
    100% |████████████████████████████████| 235kB 561kB/s ta 0:00:01


In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Adjust to your path!
sys.path += ["/home/zp489/code/bowman/jiant"]
sys.path += ["/home/zp489/code/bowman/jiant/scripts/taskmaster"]
import get_results_row as get_results_row
import grab_analysis_data as grab_analysis_data
import calculate_correlations as calculate_correlations

ModuleNotFoundError: No module named 'get_results_row'

In [4]:
all_data = grab_analysis_data.load_raw_data("/scratch/zp489/share/nlutransfer/results/phase2")

100%|██████████| 102/102 [00:00<00:00, 400.51it/s]


In [5]:
results_df = grab_analysis_data.get_correlation_raw_data(all_data)
results_df = results_df[[col for col in results_df.columns if not col.endswith("_MIX")]]

In [ ]:
# Pearson

correl, p_value, count = calculate_correlations.compute_correlation_stats(results_df)
fig = plt.figure(figsize=(16, 16))
correl_sig = correl.copy()
correl_sig[p_value > 0.05] = np.NaN
sns.heatmap(correl_sig, cmap="RdBu", vmin=-1, vmax=1)
fig.gca().xaxis.tick_top()
fig.gca().xaxis.set_label_position('top')
plt.xticks(rotation=90)

plt.axhline(10, lw=3, color="black")
plt.axvline(10, lw=3, color="black")
plt.grid()

In [ ]:
# Spearman
correl, p_value, count = calculate_correlations.compute_correlation_stats(results_df, correl_type="spearman")
fig = plt.figure(figsize=(16, 16))
correl_sig = correl.copy()
correl_sig[p_value > 0.05] = np.NaN
sns.heatmap(correl_sig, cmap="RdBu", vmin=-1, vmax=1)
fig.gca().xaxis.tick_top()
fig.gca().xaxis.set_label_position('top')
plt.xticks(rotation=90)

plt.axhline(10, lw=3, color="black")
plt.axvline(10, lw=3, color="black")
plt.grid()

In [ ]:
# Data Count
fig = plt.figure(figsize=(16, 16))
sns.heatmap(count)
fig.gca().xaxis.tick_top()
fig.gca().xaxis.set_label_position('top')
plt.xticks(rotation=90)

plt.axhline(10, lw=3, color="black")
plt.axvline(10, lw=3, color="black")
plt.grid()

In [ ]:
col_a = "record_TRG"
col_b = "se-probing-tree-depth_PRB"
results_df.plot.scatter(col_a, col_b)

In [ ]:
sub_df = results_df[[col_a, col_b]].dropna(how="any").sort_values(col_a)
print(sub_df.corr().iloc[0, 1])
sub_df

# Regression

In [ ]:
full_df = grab_analysis_data.get_regression_raw_data(
    all_data, 
    single_metadata=pd.read_csv("/scratch/zp489/share/nlutransfer/metadata/single_task_metadata.csv"),
    double_metadata=pd.read_csv("/scratch/zp489/share/nlutransfer/metadata/double_task_metadata.csv"),
)
full_df = full_df[[col for col in full_df.columns if not col.endswith("_MIX")]]

In [ ]:
regression_data_df = full_df
regression_data_df = full_df \
    .astype("float") \
    .dropna(how="all", axis=1) \
    .dropna(how="any", axis=0)  
# Drops all fully empty columns
# Drops all rows with NaNs

print(regression_data_df.shape)

In [ ]:
# 1. Include all vars

# Select X, y
x_df = regression_data_df.drop(["targ", "single_performance_mean_META_TRG", "single_performance_mean_META_INT"], axis=1)
y_srs = regression_data_df["targ"] - regression_data_df["single_performance_mean_META_TRG"]

# Adds bias / intercept column to X
full_x_df = sm.add_constant(x_df)
est = sm.OLS(y_srs, full_x_df)
est = est.fit()
print(est.summary())
y_hat = est.predict(full_x_df)

In [ ]:
# 2. Only metadata

# Select X, y
x_df = regression_data_df[[col for col in regression_data_df.columns if "META" in col and col not in [
    "single_performance_mean_META_TRG", "single_performance_mean_META_INT"]]]
y_srs = regression_data_df["targ"] - regression_data_df["single_performance_mean_META_TRG"]

# Adds bias / intercept column to X
full_x_df = sm.add_constant(x_df)
est = sm.OLS(y_srs, full_x_df)
est = est.fit()
print(est.summary())
y_hat = est.predict(full_x_df)

In [ ]:
# 3. Exclude metadata

# Select X, y
x_df = regression_data_df[[col for col in regression_data_df.columns if "META" not in col and col != "targ"]]
y_srs = regression_data_df["targ"] - regression_data_df["single_performance_mean_META_TRG"]

# Adds bias / intercept column to X
full_x_df = sm.add_constant(x_df)
est = sm.OLS(y_srs, full_x_df)
est = est.fit()
print(est.summary())
y_hat = est.predict(full_x_df)